In [ ]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [ ]:

client = RESTClient(api_key=token)

In [ ]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



In [ ]:
aggs[-1]

In [ ]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [ ]:
from heston_param import *

In [ ]:
aapl_aggs.groupby("day")

In [ ]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [ ]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [5]:
import pickle
from heston_param import *
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [6]:
while True:
	try:
		print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, daily_aggs[0]["close"].values.shape[0], 50))
		break
	except RuntimeError:
		continue

tensor([ 4.9545e+00,  8.4068e-02,  1.0327e+00, -2.0240e-01,  1.0000e-03],
       dtype=torch.float64)


In [7]:
def single_day_calibration(args):
		i, hist_volatilities, daily_agg = args
		while True:
			try:
				params = calibrate_daily_parameters(hist_volatilities[i], 0.1, day["close"].values, 0.0237, day["close"].values.shape[0], 300)
				
				return i, params
			except RuntimeError:
				continue

In [8]:
len(daily_aggs)

1273

In [9]:
daily_params = np.empty((len(daily_aggs), 5))
for i, day in enumerate((daily_aggs[:20])):
	while True:
		try:
			daily_params[i] = calibrate_daily_parameters(hist_voilatilities[i], 0.1, daily_aggs[i]["close"].values, 0.0237, daily_aggs[i]["close"].values.shape[0], 50)
			if (i + 1) % 10 == 0:
					rate =  100 *  np.round((i + 1) /len(daily_params[:20]), 2)
					print(f"{rate}% completed.")
			
			break
		except RuntimeError:
			continue



KeyboardInterrupt: 

In [10]:
def calibrate_worker(args):
	single_day_calibration(args)

In [21]:
daily_params = pickle.load(open("parameters.pkl", "rb"))

In [25]:
# runtime is 1.5 hours
daily_params.shape

array([[ 4.02208183e+00,  2.17509957e-01,  8.48127117e-01,
         5.61250460e-01,  5.04910700e-01],
       [ 6.78252317e+00,  3.29956553e-01,  1.87242133e-01,
        -6.27954532e-01,  1.00000000e-03],
       [ 6.91649100e+00,  3.22516095e-01,  6.30276744e-01,
        -4.66246111e-01,  9.24682812e-01],
       ...,
       [ 8.33918854e+00,  1.00000000e-02,  1.00000000e-01,
        -9.93558603e-01,  1.00000000e-03],
       [ 5.96346155e+00,  4.92209588e-01,  1.24595277e+00,
        -1.85387209e-01,  1.00000000e-03],
       [ 1.68325841e+00,  1.92074358e-01,  1.10428106e+00,
        -6.71438198e-01,  8.16797965e-01]])

In [16]:
from multiprocess import Pool

def calibrate_parameters_multiprocessing(daily_aggs, hist_volatilities):
    # Prepare arguments for each task
    tasks = [(i, hist_volatilities[i], daily_aggs[i]) for i in range(len(daily_aggs))]

    # Number of processes, could be set to the number of CPUs or cores
    num_processes = 4

    # Create a multiprocessing pool and map tasks to worker function
    with Pool(processes=num_processes) as pool:
        results = pool.map(calibrate_worker, tasks)

    # Process results
    daily_params = np.empty((len(daily_aggs[:20]), 5))
    for index, params in results:
        if params is not None:
            daily_params[index] = params
        else:
            print(f"Calibration failed for index {index}")

    return daily_params



In [18]:
calibrate_parameters_multiprocessing(daily_aggs[:20], hist_voilatilities[:20])

ERROR:tornado.general:SEND Error: Host unreachable


KeyboardInterrupt: 